In [1]:
from ucc_ft.checker import ft_check
from ucc_ft.codes import CatStateCode

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


This notebook demonstrates finding a circuit error on a small example. In this case, we consider a cat state preparation circuit. Although not a gadget for an error correcting code, we can still definite its fault-tolerance up to an allowed number of faults in the prepared state. 

In [2]:
num_qubits = 4
max_faults = 2
code = CatStateCode(num_qubits, max_faults)

code.stabilizers()

[stim.PauliString("+ZZ"), stim.PauliString("+_ZZ"), stim.PauliString("+__ZZ")]

In [3]:
circuit = """
OPENQASM 3.0;
include "stdgates.inc";

const uint size = __NUM_QUBITS__;
qubit[size] state;
qubit ancilla;

def cat_prep() {
    bit res = 1;
    while(res != 0) {
        reset state[0];
        res = 0;
        h state[0];

        // QASM loops supported
        for int i in [1:(size-1)] {
            reset state[i];
            cx state[0], state[i];
        }

        // Parity check
        reset ancilla;
        {   // Fails check
            cx state[1], ancilla;
            cx state[2], ancilla;
        }

        /*
        {   //passes check
            cx state[2], ancilla;
            cx state[3], ancilla;
        }*/
        res = measure ancilla;
    }
}
""".replace("__NUM_QUBITS__", str(num_qubits))

res = ft_check(code, circuit, "cat_prep", "prepare", num_ancilla=1)

print(res)

Precompiling QuantumSE...
   1474.4 ms  ✓ QuantumSE
  1 dependency successfully precompiled in 2 seconds. 65 already precompiled.


Circuit is not fault-tolerant. Fault-tolerant error locations:
[Line  19] X-Pauli error on Qubit 0
